In [1]:
# Last Revised - 2024.04.19
# Create the 'delivered' products for the given rasters (depth, wsel, polygon)

In [2]:
import os
import geopandas as gpd
import pandas as pd
from shapely.geometry import box
import rasterio
import rioxarray as rxr
from rioxarray import merge

In [3]:
# Inputs

# From raster_depth_engine ... the 'heavy' float rasters
str_depth_raster_path = r'E:\working\large_dem\07_depth_with_nan.tif'
str_wsel_raster_path = r'E:\working\large_dem\08_wsel_with_nan.tif'
str_polygon_path_gpkg = r'E:\working\large_dem\09_floodplain_ar.gpkg'

# from time_gradient script
str_flood_limits_gpkg = r'E:\sample_2d_output\hydraulic_results_1884413_wb-2410249_29-hr_14100-cfs.gpkg'

# Location to output clipped rasters
str_output_folder = r'E:\working\large_dem'

In [4]:
# Load the geopackage to get flodded limits of flowpath

gdf_flowpath_limits = gpd.read_file(str_flood_limits_gpkg, layer='01_flowpath_flooded_cells_ar')
gdf_floodplain_ar = gpd.read_file(str_polygon_path_gpkg)

In [5]:
shp_floodplain_ar = gdf_floodplain_ar.iloc[0]['geometry']

# Create an empty DataFrame
data = []
columns = ['run_name', 'flow_cfs', 'flowpath', 'clipped_geometry']
gdf_results = pd.DataFrame(data, columns=columns)

# Iterate through each row in gdf_flowpath_limits
for index, row in gdf_flowpath_limits.iterrows():
    shp_flowpath_geom = row['geometry']
    
    # Clip shp_flowpath_geom to shp_floodplain_ar
    shp_clipped_geom = shp_flowpath_geom.intersection(shp_floodplain_ar)
    
    # Handle multi-part geometries
    if shp_clipped_geom.geom_type == 'MultiLineString':
        for geom in shp_clipped_geom.geoms:
            gdf_results = pd.concat([gdf_results, pd.DataFrame([{
                'run_name': row['run_name'],
                'flow_cfs': row['flow_cfs'],
                'flowpath': row['flowpath'],
                'clipped_geometry': geom
            }])], ignore_index=True)
    else:
        gdf_results = pd.concat([gdf_results, pd.DataFrame([{
            'run_name': row['run_name'],
            'flow_cfs': row['flow_cfs'],
            'flowpath': row['flowpath'],
            'clipped_geometry': shp_clipped_geom
        }])], ignore_index=True)

gdf_results['geometry'] = gdf_results['clipped_geometry']

# Drop the 'clipped_geometry' column
gdf_results = gdf_results.drop(columns=['clipped_geometry'])

# Convert DataFrame to GeoDataFrame
gdf_results = gpd.GeoDataFrame(gdf_results, geometry='geometry')

# Set the coordinate reference system of new geodataframe
gdf_results.crs = gdf_floodplain_ar.crs

In [6]:
gdf_floodplain_by_flowpath = gdf_results

# add this layer... to gdf_flowpath_limits

In [7]:
# convert entire depth grid and wsel grid to integers - 3m resolution - EPSG:3857

# --- Output raster standards ---
str_output_crs = 'EPSG:3857'
flt_desired_res = 3.0 #3 meter cell resolution
# ------

# convert the depth DEM to InFRM compliant rasters
# Create a raster for the entire run's depth
with rxr.open_rasterio(str_depth_raster_path) as xds_raw_dem:
     # reproject the DEM to the requested CRS
    xds_reproject = xds_raw_dem.rio.reproject(str_output_crs)
    
    # change the depth values to integers representing 1/10th interval steps
    # Example - a cell value of 25 = a depth of 2.5 units (feet or meters)
    xds_reproject_scaled = ((xds_reproject * 10) + 0.5) // 1
    
    # set the n/a cells to a value of 65535
    xds_reproject_scaled = xds_reproject_scaled.fillna(65535)
    
    # set the nodata value to 65535 - InFRM compliant
    xds_reproject_scaled = xds_reproject_scaled.rio.set_nodata(65535)
    
    # using the merge on a single raster to allow for user supplied
    # raster resolution of the output
    xds_depth_raster = rxr.merge.merge_arrays(xds_reproject_scaled,
                                                     res=(flt_desired_res),
                                                     nodata=(65535))

C:\Users\civil\anaconda3\envs\tx-bridge\lib\site-packages\rioxarray\raster_writer.py:115: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


In [8]:
# Create a raster for the entire run's water surface elevation raster
with rxr.open_rasterio(str_wsel_raster_path) as xds_raw_dem:
     # reproject the DEM to the requested CRS
    xds_reproject = xds_raw_dem.rio.reproject(str_output_crs)
    
    # change the wsel values to integers representing 1/10th interval steps
    # Example - a cell value of 25 = a depth of 2.5 units (feet or meters)
    xds_reproject_scaled = ((xds_reproject * 10) + 0.5) // 1
    
    # set the n/a cells to a value of 65535
    xds_reproject_scaled = xds_reproject_scaled.fillna(65535)
    
    # set the nodata value to 65535 - InFRM compliant
    xds_reproject_scaled = xds_reproject_scaled.rio.set_nodata(65535)
    
    # using the merge on a single raster to allow for user supplied
    # raster resolution of the output
    xds_wsel_raster = rxr.merge.merge_arrays(xds_reproject_scaled,
                                                     res=(flt_desired_res),
                                                     nodata=(65535))

C:\Users\civil\anaconda3\envs\tx-bridge\lib\site-packages\rioxarray\raster_writer.py:115: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


In [9]:
# Clip the depth and wsel raster to to the polygons in gdf_flowpath_limits

# Reproject gdf_results to str_output_crs
gdf_flowpath_limits_3857 = gdf_flowpath_limits.to_crs(str_output_crs)

list_depth_rasters = []
list_wsel_rasters = []

for index, row in gdf_flowpath_limits_3857.iterrows():
    
    geom = row['geometry']
    
    # ----- create clipped depth rasters-----
    xds_clipped_depth = xds_depth_raster.rio.clip([geom])
    str_depth_name = 'depth_' + row['flowpath'] + '_' + str(row['flow_cfs']) + '.tif'
    str_depth_filepath = os.path.join(str_output_folder, str_depth_name)
    
    # compress and write out depth raster as unsigned 16 bit integer
    xds_clipped_depth.rio.to_raster(str_depth_filepath,compress='lzw',dtype="uint16")
    
    list_depth_rasters.append(str_depth_filepath)
    
    # ----- create clpped wsel rasters -----
    xds_clipped_wsel = xds_wsel_raster.rio.clip([geom])
    str_wsel_name = 'wsel_' + row['flowpath'] + '_' + str(row['flow_cfs']) + '.tif'
    str_wsel_filepath = os.path.join(str_output_folder, str_wsel_name)
    
    # compress and write out wsel raster as signed 32 bit integer
    xds_clipped_wsel.rio.to_raster(str_wsel_filepath,compress='lzw',dtype="int32")
    
    list_wsel_rasters.append(str_wsel_filepath)

In [10]:
#str_flood_limits_gpkg
# Contains:  
#   00_hec_info (information about the simulation run)
#   01_flowpath_flooded_cells (polygon of the merged wet cells around a flowpath, with buffer)
#   02_cells_wsel (wet cells per this run, including hours to stability)

gdf_results['depth_raster_path'] = list_depth_rasters
gdf_results['wsel_raster_path'] = list_wsel_rasters

# Append the floodplain limits in 'gdf_results' to existing geopackage
gdf_results.to_file(str_flood_limits_gpkg, layer='04_floodplain_limits', driver='GPKG', append=True)